# Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

**Resilient:** RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

**Distributed:** The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

**Dataset:** The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 

Key Characteristics of RDDs:

- **Immutable:** Once an RDD is created, it cannot be modified. 
- **Lazily Evaluated:** RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- **In-Memory:** The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.


### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
nums = list(range(1,1001))
len(nums)

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">

To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]', 'RDD practice')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(nums, numSlices=10)
print(type(rdd))

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [4]:
rdd.getNumPartitions()

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [5]:
rdd.count()

1000

In [6]:
rdd.first()

1

In [7]:
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [8]:
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [9]:
## Note: When you are dealing with big data, this could make your computer crash! It's best to avoid using the collect() method
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [10]:
import numpy as np

np.random.seed(1)
nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([4.17022005e-01, 1.44064899e+00, 3.43124452e-04, 1.20933029e+00,
       7.33779454e-01, 5.54031569e-01, 1.30382148e+00, 2.76448582e+00,
       3.57090727e+00, 5.38816734e+00, 4.61113966e+00, 8.22263400e+00,
       2.65787925e+00, 1.22936441e+01, 4.10813898e-01, 1.07274802e+01,
       7.09418164e+00, 1.00564169e+01, 2.66735183e+00, 3.96202978e+00,
       1.68156359e+01, 2.13017547e+01, 7.20875610e+00, 1.66157428e+01,
       2.19097288e+01, 2.32597733e+01, 2.29619371e+00, 1.09353393e+00,
       4.92508217e+00, 2.63442751e+01, 3.04875185e+00, 1.34754440e+01,
       3.16103545e+01, 1.81276197e+01, 2.42156990e+01, 1.13585627e+01,
       2.54005343e+01, 3.17157755e+01, 7.13242816e-01, 3.00057726e+01,
       4.05433046e+01, 3.14229575e+01, 1.20590917e+01, 3.47282905e+01,
       4.64517030e+00, 2.06031022e+01, 4.27039886e+01, 1.40934791e+01,
       1.41009916e+01, 6.50142861e+00, 9.87714851e-01, 3.52994477e+01,
       1.12162901e+01, 1.43395196e+01, 2.70365238e+01, 2.98830253e+00,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [11]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)

[0.417022004702574,
 1.4406489868843162,
 0.0003431244520346599,
 1.209330290527359]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied. Assume a sales tax of 8%, i.e. that the current price data is actually 108% of what it should be.

To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `num`: (float) number to be multiplied by the sales tax.

Apply that function to the RDD by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [12]:
def sales_tax(num):
    return num / 1.08

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` RDD.

In [13]:
revenue_minus_tax.take(10)

[0.38613148583571666,
 1.3339342471151074,
 0.0003177078259580184,
 1.119750269006814,
 0.6794254204495974,
 0.5129921931599878,
 1.2072421107812004,
 2.559709089207761,
 3.306395618588916,
 4.989043833364416]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [14]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [15]:
discounted.take(10)

[0.347518337252145,
 1.2005408224035967,
 0.0002859370433622166,
 1.0077752421061326,
 0.6114828784046377,
 0.461692973843989,
 1.0865178997030804,
 2.303738180286985,
 2.9757560567300243,
 4.490139450027975]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [16]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[808.7548791977921,
 795.5962541991298,
 784.6868851954209,
 756.8310401687224,
 753.4277907153834,
 752.0937136346447,
 747.1368663633596,
 737.66773326942,
 730.5313349924388,
 725.1608962422889,
 720.4254585887082,
 718.7262188305615,
 716.2342491392883,
 712.8320720999051,
 711.8595792911022]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [17]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:274 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [18]:
mapped = price_items.map(lambda x: (x, x / 1.08 * 0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.417022004702574, 0.347518337252145), (1.4406489868843162, 1.2005408224035967), (0.0003431244520346599, 0.0002859370433622166), (1.209330290527359, 1.0077752421061326), (0.7337794540855652, 0.6114828784046377), (0.5540315686127868, 0.461692973843989), (1.3038214796436964, 1.0865178997030804), (2.764485816344382, 2.303738180286985), (3.5709072680760294, 2.9757560567300243), (5.3881673400335695, 4.490139450027975)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [19]:
flat_mapped = price_items.flatMap(lambda x : (x, x / 1.08 * 0.9 ))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.417022004702574, 0.347518337252145, 1.4406489868843162, 1.2005408224035967, 0.0003431244520346599, 0.0002859370433622166, 1.209330290527359, 1.0077752421061326, 0.7337794540855652, 0.6114828784046377]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than &#36;300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:

* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [20]:
selected_items = discounted.filter(lambda x: x > 300)
selected_items.count()

272

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [21]:
selected_items.reduce(lambda x, y: x + y)

123432.4660522227

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [22]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(8, 303.9717333600381),
 (36, 315.8957533626659),
 (29, 320.6109689548023),
 (18, 320.58739183867544),
 (48, 329.4565433918027),
 (36, 352.7343621192842),
 (4, 336.5077558595325)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [23]:
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(10, 2505.4128261985734),
 (30, 3382.449750500504),
 (20, 2162.532818544579),
 (40, 2733.835414659196),
 (50, 1009.1271615645499),
 (41, 2288.6328380275563),
 (21, 1511.275378505312),
 (1, 2752.542147304854),
 (31, 3056.504120650862),
 (11, 1511.8044372912896)]

In [24]:
total_spent.sortBy(lambda x: x[1], ascending = False).collect()

[(37, 6058.32282961381),
 (35, 5147.3165498027965),
 (14, 4718.853598241101),
 (3, 4577.11556237689),
 (32, 3661.5914194018924),
 (39, 3590.199456355873),
 (34, 3584.8861644479093),
 (4, 3450.7117495281045),
 (30, 3382.449750500504),
 (42, 3314.794549097422),
 (19, 3258.7620596071074),
 (22, 3200.1246013611844),
 (31, 3056.504120650862),
 (36, 3053.623359721479),
 (45, 3010.218962331668),
 (8, 2912.0427078664316),
 (46, 2837.3218299486853),
 (44, 2832.057231700557),
 (1, 2752.542147304854),
 (40, 2733.835414659196),
 (13, 2699.181298186921),
 (26, 2663.0870131901474),
 (2, 2634.2461944306783),
 (10, 2505.4128261985734),
 (17, 2474.19078593852),
 (12, 2380.99877057044),
 (41, 2288.6328380275563),
 (29, 2271.601948461445),
 (49, 2206.520353564143),
 (20, 2162.532818544579),
 (16, 2070.5342988892085),
 (5, 1988.2251520732625),
 (24, 1926.9060030801184),
 (33, 1899.7417139575373),
 (48, 1717.7722170913798),
 (43, 1653.3750643830203),
 (28, 1639.9325449739288),
 (23, 1593.6702466360014),
 (

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [25]:
sorted(sales_data.countByKey().items(), key=lambda kv:kv[1], reverse=True)

[(49, 11),
 (22, 11),
 (11, 10),
 (23, 10),
 (28, 9),
 (13, 8),
 (38, 8),
 (44, 8),
 (18, 8),
 (35, 8),
 (48, 8),
 (2, 7),
 (43, 7),
 (46, 7),
 (37, 7),
 (20, 7),
 (32, 7),
 (47, 7),
 (17, 7),
 (8, 6),
 (40, 6),
 (16, 5),
 (4, 5),
 (33, 5),
 (6, 5),
 (26, 5),
 (34, 5),
 (5, 5),
 (36, 5),
 (30, 4),
 (24, 4),
 (42, 4),
 (9, 4),
 (31, 4),
 (19, 4),
 (27, 4),
 (25, 4),
 (21, 3),
 (10, 3),
 (50, 3),
 (41, 3),
 (3, 3),
 (7, 3),
 (29, 3),
 (14, 2),
 (1, 2),
 (45, 2),
 (12, 2),
 (39, 2),
 (15, 2)]

### Stop the SparkContext

Now that we are finished with our analysis, stop the `sc`.

In [26]:
sc.stop()

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
